In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import glob
from tidegauge_functions import read_GPS_SONEL

%matplotlib inline

In [2]:
datum = '?????'
ext = '.neu'
dir_in = 'data/GPS'

In [3]:
pattern = os.path.join(dir_in, 'd*' + datum + ext)
print(pattern)

data/GPS\d*?????.neu


In [4]:
filenames = sorted(glob.glob(pattern))  # , key=alphanum_key)

In [ ]:
for f, filepath in enumerate(filenames):
    df = read_GPS_SONEL(filepath)
    print(f'\n\n{filepath}')
    print(df.head())

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
def decompose(df, var, site, period):  
    
    decomposition = seasonal_decompose(df[var], freq = period)

    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    Amplitude = max(seasonal) - min(seasonal)
    
    decomposition.plot();
    df['denoised'] = df[var] - residual
    df['trend'] = trend
    print(f'\n\n{filepath}')
    print(df.head())
    print(f'{site} Seasonal Amplitude =', Amplitude)
    seasonal_amp = pd.DataFrame(list(zip(list({site}), list(Amplitude))),
                            colums = ['Site_Name', 'Seasonal_Amplitude'])
    seasonal_amp.to_csv('GPS_Seasonal_Amplitude_SONEL')

# Change Point Detection
def change_pt_detection(df, var, points):
    model = "l1"  
    algo = rpt.Dynp(model=model, min_size=3, jump=5).fit(points)
    my_bkps = algo.predict(n_bkps=10)

    
def change_pt_plot()
    rpt.show.display(points, my_bkps, figsize=(10, 6))
    plt.suptitle(f"{site}Change Point Detection")
    plt.show()

In [5]:
def test_calc_OLS(df, var):
    
    x, y = np.arange(len(df[var].dropna())), df[var].dropna()
    x = sm.add_constant(x)
    model = sm.OLS(y, x)
    res = model.fit()
    return res

def test_plot_OLS(df, res, site, var, trend_array):
    fig, ax = plt.subplots(1, 1, figsize=(12,6));
    ax.plot(df[var].dropna().index, df[var].dropna().values, 
            label='trend', marker='.', linestyle='')
    ax.plot(df[var].dropna().index, [res.params.x1*i + res.params.const for i in np.arange(len(df[var].dropna()))])
    # ax[1].plot(df['Vertical'].dropna().index, res.resid.values);
    # ax[1].plot(df['Vertical'].dropna().index, np.abs(res.resid.values));
    # ax[1].hlines(0, 0, len(res.resid), color='k');
    ax.set_title(f"Trend = {trend_array * 1:.2f} mm/yr");
    # ax[1].set_title("Residuals");
    plt.suptitle(f"{site}")
    plt.savefig(f'figs/test_GPS_OLS_{site}.png')
    
    

In [ ]:
for f, filepath in enumerate(filenames):
    print(f'\n\n{filepath}')
    print(f'f: {f}')
    df = read_GPS_SONEL(filepath)
    df = decompose(df, 'Vertical', filepath[-24:-20], 365)
    

In [6]:
def convert_trend_toyearly(df, res):
    period = df.index.year.value_counts().max()
    yearlytrend = res.params.x1 * period
    
    return yearlytrend

In [7]:
sonel_trend_array = np.full(np.shape(filenames)[0], np.nan)
site_name_array = []


for f, filepath in enumerate(filenames):
    print(f'\n\n{filepath}')
    print(f'f: {f}')
    ## ToDo - extract and keep other important thing about each site from the file...Lat/Lon, name, etc.
    
    # Read in data
    df = read_GPS_SONEL(filepath)
    
    
    # Get trend using linear regression
    res = test_calc_OLS(df, 'Vertical')
    sonel_trend_array[f] = convert_trend_toyearly(df, res)
    
    
    # Save site name
    site_name_array.append(filepath[-24:-20])
    
   # print(f"Trend = {sonel_trend_array[f] * 1:.2f} mm/yr")
    
#     print(res.summary())
   # np.savetxt('GPS_trend.csv', {sonel_trend_array[f] * 365:.2f}, delimiter=',')
    # Make Plot
    #test_plot_OLS(df, res, filepath[-24:-20], 'Vertical', sonel_trend_array[f])
    
    



data/GPS\dAB44_99999X006_ULR6B.neu
f: 0


data/GPS\dAB48_XXXXXXXXX_JPL14.neu
f: 1


data/GPS\dAB50_99999X007_ULR6B.neu
f: 2


data/GPS\dAKSE_XXXXXXXXX_NGL14.neu
f: 3


data/GPS\dAKSI_XXXXXXXXX_NGL14.neu
f: 4


data/GPS\dANC2_XXXXXXXXX_NGL14.neu
f: 5


data/GPS\dCHUR_40128M002_ULR6B.neu
f: 6


data/GPS\dEYAC_99999X043_ULR6B.neu
f: 7


data/GPS\dJNU1_49519S001_NGL14.neu
f: 8


data/GPS\dKEN5_XXXXXXXXX_NGL14.neu
f: 9


data/GPS\dNUUK_43013M001_NGL14.neu
f: 10


data/GPS\dQAQ1_43007M001_ULR6B.neu
f: 11


data/GPS\dSCOR_43006M002_ULR6B.neu
f: 12


data/GPS\dTBON_XXXXXXXXX_NGL14.neu
f: 13


data/GPS\dTHU1_43001M001_ULR6B.neu
f: 14


data/GPS\dTHU2_43001M002_ULR6B.neu
f: 15


data/GPS\dTHU3_43001M002_ULR6B.neu
f: 16


In [ ]:
str(filenames)

In [8]:
def get_SONEL_Lat(filenames):
    with open(filenames, "r") as f:
        for line in f:
            if 'Latitude' in line:
                lat = line.split(':')[1].split(' ')[3]
    return lat

In [9]:
def get_SONEL_Lon(filenames):
    with open(filenames, "r") as f:
        for line in f:
            if 'Longitude' in line:
                lon = line.split(':')[1].split(' ')[3]

    return lon

In [10]:
def get_SONEL_Height(filenames):
    with open(filenames, "r") as f:
        for line in f:            
            if 'Height' in line:
                height = line.split(':')[1].split(' ')[3]
    
    return height

In [13]:
sonel_Lat_array = np.full(np.shape(filenames)[0], np.nan)
sonel_Lon_array = np.full(np.shape(filenames)[0], np.nan)
for f, filepath in enumerate(filenames):
    sonel_Lat_array[f] = get_SONEL_Lat(filenames[f])
    sonel_Lon_array[f] = get_SONEL_Lon(filenames[f])
    sonel_Height_array[f] = get_SONEL_Height(filenames[f])


ValueError: could not convert string to float: 

In [ ]:
#Write CSV trend array
columns = ['SITE', 'LAT', 'LON', 'Height', 'TREND_Vertical']
out_array = np.array([site_name_array, sonel_Lat_array, sonel_Lon_array,sonel_Height_array, sonel_trend_array])

df = pd.DataFrame(data=out_array.T, columns=columns) #, index=time)
df = df.apply(pd.to_numeric, errors='ignore')
# df.index = pd.to_datetime(trajectory.index)
df.to_csv('data/trends_OLS_SONEL.csv')  # , na_rep=" ")